In [1]:
import os
import asyncio
from typing import Optional
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough

c:\Users\xgear\miniconda3\envs\intern\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
from langchain_ollama import ChatOllama, OllamaEmbeddings

# LLM cho chat
llm = ChatOllama(
    model="gemma3:1b",
    base_url="http://localhost:11434"
)

# Embedding model
embedding = OllamaEmbeddings(
    model="embeddinggemma:300m",
    base_url="http://localhost:11434"
)

In [18]:
summarize_chain: Runnable = (
    ChatPromptTemplate.from_messages([
        ("system", "Summarize the following topic concisely:"),
        ("user", "{topic}")
    ])
    | llm
    | StrOutputParser()
)


In [19]:
questions_chain: Runnable = (
    ChatPromptTemplate.from_messages([
        ("system", "Generate three interesting questions about the following topic:"),
        ("user", "{topic}")
    ])
    | llm
    | StrOutputParser()
)

In [20]:
terms_chain: Runnable = (
    ChatPromptTemplate.from_messages([
        ("system", "Identify 5-10 key terms from the following topic, separated by commas:"),
        ("user", "{topic}")
    ])
    | llm
    | StrOutputParser()
)

In [21]:
map_chain = RunnableParallel({
    "summary": summarize_chain,
    "questions": questions_chain,
    "key_terms": terms_chain,
    "topic": RunnablePassthrough(), # Pass the original topic through
})

In [22]:
synthesis_prompt = ChatPromptTemplate.from_messages([
    ("system", """Based on the following information:
    Summary: {summary}
    Related Questions: {questions}
    Key Terms: {key_terms}
    Synthesize a comprehensive answer."""),
    ("user", "Original topic: {topic}")
])

In [23]:
full_parallel_chain = map_chain | synthesis_prompt | llm | StrOutputParser()

In [24]:
async def run_parallel_example(topic: str) -> None: 
   """ 
   Asynchronously invokes the parallel processing chain with a 
specific topic 
   and prints the synthesized result. 
 
   Args: 
       topic: The input topic to be processed by the LangChain 
chains. 
   """ 
   if not llm: 
       print("LLM not initialized. Cannot run example.") 
       return 
 
   print(f"\n--- Running Parallel LangChain Example for Topic: '{topic}' ---") 
   try: 
       # The input to `ainvoke` is the single 'topic' string,  
       # then passed to each runnable in the `map_chain`. 
       response = await full_parallel_chain.ainvoke(topic) 
       print("\n--- Final Response ---") 
       print(response) 
   except Exception as e: 
       print(f"\nAn error occurred during chain execution: {e}") 

In [25]:
test_topic = "The history of space exploration" 

In [ ]:
await run_parallel_example(test_topic)


--- Running Parallel LangChain Example for Topic: 'The history of space exploration' ---

--- Final Response ---
Okay, let's synthesize a comprehensive answer about the history of space exploration, building on the information we've established. Here’s a detailed overview, touching on key periods, motivations, and lasting impacts:

**The Genesis of Exploration: From Theoretical Dreams to Robotic Missions (1903-1957)**

The story of space exploration begins long before rockets and satellites. Early observations by pioneers like Konstantin Tsiolkovsky (credited with developing the rocket equation) and Robert Goddard demonstrated the potential for human space travel. However, it wasn't until the 1930s that the United States and the Soviet Union began serious efforts to reach beyond Earth's atmosphere.

* **Early Years (1903-1957):** The initial impetus was primarily driven by military necessity.  The launch of the German V-2 rocket in 1942, though unreliable, proved the concept of ballis

: 